# 1. Libraries

In [3]:
# Import Libraries
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# 2. Introduction

The purpose of this notebook is to gather data on the European ARMS Challenge (EUAC) tournament series. <br>
The EUAC is an online biweekly tournament series for the video game "ARMS" released for the Nintendo Switch console in 2017. <br>
The EUAC started in 2017 and ran consistently until 2022.<br>
<br>
More information can be found in references below.

This notebook is to purely gather the data, act as a proof of concept, and format it for future use/analysis. <br>
It will then be exported to a csv file if successful. <br>

# 3. Objectives

- Determine what data that is necessary
- Gather EUAC#1 data from start.gg using their api
- Gather EUAC#2 to EUAC#110 data from Challonge.com using their api
- Format the data into a pandas dataframe
- Export to csv

## 3.1. Objective 1

### Determine what data is necessary

For each tournament, we will need: <br>
- The participants who entered
- Each match that was played
- The result of each match
- The date the tournament took place
- The final placements
- Each player's seeding

## 3.2. Objective 2 - Data Gathering

The tournaments were mostly hosted on Challonge.com but the first one was hosted on start.gg. <br>

### 3.2.1. - Start.gg

Start.gg's API uses GraphQL as its query language <br>
The GraphQL queries will be wrapped in multiline strings so they can be sent to the start.gg API to retrieve the data. <br>
Queries also expect an API token to passed through the header. This token will be loaded from a .env file.<br>
<br>
A link to the tutorial that was followed to come up with the queries can be found in References.

In [4]:
# Load API Keys stored in .env file
load_dotenv(".env")
START_GG_API_TOKEN = os.getenv("START_GG_API_TOKEN")

In [5]:
query = """
query TournamentQuery {
  tournament(slug: "tournament/eu-arms-challenge-1") {
    name
    events {
      name
      entrants {
        nodes {
          name
        }
      }
    }
  }
}
"""

In [6]:
url = "https://api.start.gg/gql/alpha"
headers = {
    "Authorization": f"Bearer {START_GG_API_TOKEN}",
    "Content-Type": "application/json"
}
response = requests.post(url, json={'query': query}, headers=headers)
data = response.json()

In [7]:
data

{'data': {'tournament': {'name': 'EU ARMS Challenge #1',
   'events': [{'name': 'EU ARMS Challenge #1',
     'entrants': {'nodes': [{'name': 'Owdy'},
       {'name': 'TCM | Raffa'},
       {'name': 'FR | Maxou0708'},
       {'name': 'FrankTank'},
       {'name': 'Altair'},
       {'name': 'TCM | BD82'},
       {'name': 'Octey | Octey'},
       {'name': 'Kotorious BRD'},
       {'name': 'BambooBoss'},
       {'name': 'FR|TCM | InkAlyut'},
       {'name': 'RD | | Dushni'},
       {'name': 'Alumento'},
       {'name': 'Beannie'},
       {'name': 'Gleeface'},
       {'name': 'Sabaca'},
       {'name': 'SC☆Momso'},
       {'name': 'Rapha_MTH'},
       {'name': 'Raayo'},
       {'name': 'VilleViljar'},
       {'name': 'Orthuss'},
       {'name': 'ocrim'},
       {'name': 'Linkirto'}]}}]}},
 'extensions': {'cacheControl': {'version': 1,
   'hints': [{'path': ['tournament'], 'maxAge': 300, 'scope': 'PRIVATE'}]},
  'queryComplexity': 2},
 'actionRecords': []}

Data is a nested dictionary. Now we'll gather a list of participants by digging through this

In [55]:
data["data"]["tournament"]["name"]

'EU ARMS Challenge #1'

In [8]:
data["data"]["tournament"]["events"][0]["entrants"]

{'nodes': [{'name': 'Owdy'},
  {'name': 'TCM | Raffa'},
  {'name': 'FR | Maxou0708'},
  {'name': 'FrankTank'},
  {'name': 'Altair'},
  {'name': 'TCM | BD82'},
  {'name': 'Octey | Octey'},
  {'name': 'Kotorious BRD'},
  {'name': 'BambooBoss'},
  {'name': 'FR|TCM | InkAlyut'},
  {'name': 'RD | | Dushni'},
  {'name': 'Alumento'},
  {'name': 'Beannie'},
  {'name': 'Gleeface'},
  {'name': 'Sabaca'},
  {'name': 'SC☆Momso'},
  {'name': 'Rapha_MTH'},
  {'name': 'Raayo'},
  {'name': 'VilleViljar'},
  {'name': 'Orthuss'},
  {'name': 'ocrim'},
  {'name': 'Linkirto'}]}

In [13]:
data["data"]["tournament"]["events"][0]["entrants"]["nodes"][1]

{'name': 'TCM | Raffa'}

In [14]:
data["data"]["tournament"]["events"][0]["entrants"]["nodes"][1]["name"]

'TCM | Raffa'

In [58]:
# Testing basic for loop
for i in data["data"]["tournament"]["events"][0]["entrants"]["nodes"]:
    print(i["name"])

Owdy
TCM | Raffa
FR | Maxou0708
FrankTank
Altair
TCM | BD82
Octey | Octey
Kotorious BRD
BambooBoss
FR|TCM | InkAlyut
RD | | Dushni
Alumento
Beannie
Gleeface
Sabaca
SC☆Momso
Rapha_MTH
Raayo
VilleViljar
Orthuss
ocrim
Linkirto


In [16]:
playerArray = []
for i in data["data"]["tournament"]["events"][0]["entrants"]["nodes"]:
    playerArray.append(i["name"])

In [18]:
playerArray

['Owdy',
 'TCM | Raffa',
 'FR | Maxou0708',
 'FrankTank',
 'Altair',
 'TCM | BD82',
 'Octey | Octey',
 'Kotorious BRD',
 'BambooBoss',
 'FR|TCM | InkAlyut',
 'RD | | Dushni',
 'Alumento',
 'Beannie',
 'Gleeface',
 'Sabaca',
 'SC☆Momso',
 'Rapha_MTH',
 'Raayo',
 'VilleViljar',
 'Orthuss',
 'ocrim',
 'Linkirto']

All players gathered. <br>To note for later: players can sign up with "tags". Denoted with a |. But | can be present in a tag

### 3.2.2 - Challonge.com

# References

- https://armswiki.org/wiki/EU_ARMS_Challenge
- https://developer.start.gg/docs/examples/queries/get-event